In [1]:
import pandas as pd
# 库文件
import numpy as np
import matplotlib.pyplot as plt
import warnings
warnings.filterwarnings("ignore", category=UserWarning)
warnings.filterwarnings("ignore", category=FutureWarning)
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from skopt import gp_minimize
from sklearn.svm import SVR
from sklearn.metrics import mean_squared_error
from scipy.optimize import minimize
from sklearn.metrics.pairwise import rbf_kernel
import pandas as pd
import tensorflow as tf
import torch
import torch.nn as nn
import torch.optim as optim
import matplotlib.pyplot as plt
from matplotlib.font_manager import FontProperties
# 设置中文字体
font = FontProperties(fname=r"c:\windows\fonts\simsun.ttc", size=12)  # 替换为你的中文字体文件路径


In [2]:
# 替换成你的Excel文件路径
file_path = f'C:\\Users\\haokw\\Documents\\GitHub\\VScode\\z综合信息\\数据\\123.xlsx'
df1 = pd.read_excel(file_path,sheet_name='Sheet1')
print('1')


1


In [3]:
df = pd.concat([df1],axis=0)
df.info()


<class 'pandas.core.frame.DataFrame'>
RangeIndex: 330585 entries, 0 to 330584
Data columns (total 36 columns):
 #   Column    Non-Null Count   Dtype         
---  ------    --------------   -----         
 0   时间        330585 non-null  datetime64[ns]
 1   富氧率       330585 non-null  float64       
 2   透气性指数     330585 non-null  float64       
 3   CO        330585 non-null  float64       
 4   H2        330585 non-null  float64       
 5   CO2       330585 non-null  float64       
 6   标准风速      330585 non-null  int64         
 7   富氧流量      330585 non-null  int64         
 8   冷风流量      330585 non-null  float64       
 9   鼓风动能      330585 non-null  int64         
 10  炉腹煤气量     330585 non-null  int64         
 11  炉腹煤气指数    330585 non-null  float64       
 12  理论燃烧温度    330585 non-null  int64         
 13  顶压        330585 non-null  float64       
 14  顶压2       330585 non-null  float64       
 15  顶压3       330585 non-null  float64       
 16  顶压4       330585 non-null  float64    

In [4]:
# 使用rename方法修改列名
df_params = df.copy()
df_params.rename(columns={'时间': '主参数时间戳'}, inplace=True)
df_processed = pd.DataFrame()


In [5]:
# 分批处理转成时间按戳
total_entries = len(df_params)
num_chunks = total_entries // 100000 + 1
for i in range(num_chunks):
    start_idx = i * 100000
    end_idx = min((i + 1) * 100000, total_entries)
    
    # 获取子集
    subset_df = df_params.iloc[start_idx:end_idx]

    # 定义起始时间
    start_time ='2023-02-09 00:00:00'
    start_time_stamp = pd.Timestamp(start_time).timestamp()

    # 转换为时间戳并保存到新列
    params_timestamp  = subset_df.iloc[:, 0].apply(lambda x: pd.Timestamp(x).timestamp())

    # 在后续步骤中使用新列进行进一步的操作
    sampling_time = 1
    subset_df['主参数时间戳2'] = (params_timestamp - start_time_stamp) / sampling_time
    df_processed = pd.concat([df_processed,subset_df],axis=0)
    print(i,num_chunks)


C:\Users\haokw\AppData\Local\Temp\ipykernel_18116\3849187168.py:20: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  subset_df['主参数时间戳2'] = (params_timestamp - start_time_stamp) / sampling_time
C:\Users\haokw\AppData\Local\Temp\ipykernel_18116\3849187168.py:20: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  subset_df['主参数时间戳2'] = (params_timestamp - start_time_stamp) / sampling_time


0 4
1 4
2 4
3 4


C:\Users\haokw\AppData\Local\Temp\ipykernel_18116\3849187168.py:20: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  subset_df['主参数时间戳2'] = (params_timestamp - start_time_stamp) / sampling_time
C:\Users\haokw\AppData\Local\Temp\ipykernel_18116\3849187168.py:20: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  subset_df['主参数时间戳2'] = (params_timestamp - start_time_stamp) / sampling_time


In [6]:
df_processed.columns


Index(['主参数时间戳', '富氧率', '透气性指数', 'CO', 'H2', 'CO2', '标准风速', '富氧流量', '冷风流量',
       '鼓风动能', '炉腹煤气量', '炉腹煤气指数', '理论燃烧温度', '顶压', '顶压2', '顶压3', '顶压4', '富氧压力',
       '冷风压力', '冷风压力2', '全压差', '热风压力', '热风压力2', '实际风速', '冷风温度', '热风温度', '顶温东北',
       '顶温西南', '顶温西北', '顶温东南', '顶温下降管', '阻力系数', '鼓风湿度', '设定喷煤量', '本小时实际喷煤量',
       '上小时实际喷煤量', '主参数时间戳2'],
      dtype='object')

In [7]:
input_term = ['主参数时间戳', '主参数时间戳2','富氧率', '透气性指数', 'CO', 'H2', 'CO2', '标准风速', '富氧流量', '冷风流量',
            '鼓风动能', '炉腹煤气量', '炉腹煤气指数', '理论燃烧温度', '顶压', '顶压2', '顶压3', '顶压4', '富氧压力',
            '冷风压力', '冷风压力2', '全压差', '热风压力', '热风压力2', '实际风速', '冷风温度', '热风温度', '顶温东北',
            '顶温西南', '顶温西北', '顶温东南', '顶温下降管', '阻力系数', '鼓风湿度', '设定喷煤量', '本小时实际喷煤量',
            '上小时实际喷煤量']


In [9]:
df_processed['主参数时间戳2'][1]


50416.0

In [12]:
# 间隔60s均值
index = np.arange(15*60,934*60,1)
len(index)

end_idx = 15*3600
time = 60
dataframe = pd.DataFrame()

for i in range(len(index)-1):
    # if i < 7360:
    #     continue
    print(i,index[i])
    start_idx = df_processed[df_processed['主参数时间戳2'] < index[i]*time].index[-1]
    # print(start_idx,df_processed['主参数时间戳2'][start_idx])
    end_idx = df_processed[df_processed['主参数时间戳2'] < index[i+1]*time].index[-1]
    # print(end_idx,df_processed['主参数时间戳2'][end_idx])
    # 获取子集
    subset_df = df_processed.iloc[start_idx:end_idx]
    # print(subset_df.head(2))
    # print(subset_df.tail(2))
    # print('fdsahdrhgdfx hfxthdfghy j gh')
    subset_df_processed = pd.DataFrame(subset_df[input_term].mean()).T
    subset_df_processed['主参数时间戳2']=index[i+1]*time
    # print(subset_df_processed)
    dataframe = pd.concat([dataframe,subset_df_processed],axis=0)
    


    
print(dataframe)
new_sequence_1h_mean = dataframe.copy()
# 将 df_combined 添加到新的 ExcelWriter 中
with pd.ExcelWriter(f"C:\\Users\\haokw\\Documents\\GitHub\\gaolu\\MPC\\高炉\\0数据处理\\新输入输出模式\\new\\间隔1min平均.xlsx", engine='xlsxwriter') as writer:
    # 将其他 sheet 写入新文件
    new_sequence_1h_mean.to_excel(writer, sheet_name='1h_mean', index=False)


0 900
1 901
2 902
3 903
4 904
5 905
6 906
7 907
8 908
9 909
10 910
11 911
12 912
13 913
14 914
15 915
16 916
17 917
18 918
19 919
20 920
21 921
22 922
23 923
24 924
25 925
26 926
27 927
28 928
29 929
30 930
31 931
32 932
33 933
34 934
35 935
36 936
37 937
38 938
39 939
40 940
41 941
42 942
43 943
44 944
45 945
46 946
47 947
48 948
49 949
50 950
51 951
52 952
53 953
54 954
55 955
56 956
57 957
58 958
59 959
60 960
61 961
62 962
63 963
64 964
65 965
66 966
67 967
68 968
69 969
70 970
71 971
72 972
73 973
74 974
75 975
76 976
77 977
78 978
79 979
80 980
81 981
82 982
83 983
84 984
85 985
86 986
87 987
88 988
89 989
90 990
91 991
92 992
93 993
94 994
95 995
96 996
97 997
98 998
99 999
100 1000
101 1001
102 1002
103 1003
104 1004
105 1005
106 1006
107 1007
108 1008
109 1009
110 1010
111 1011
112 1012
113 1013
114 1014
115 1015
116 1016
117 1017
118 1018
119 1019
120 1020
121 1021
122 1022
123 1023
124 1024
125 1025
126 1026
127 1027
128 1028
129 1029
130 1030
131 1031
132 1032
133 1033
134 

In [ ]:
print(dataframe)
new_sequence_1h_mean = dataframe.copy()
# 将 df_combined 添加到新的 ExcelWriter 中
with pd.ExcelWriter(f"C:\\Users\\haokw\\Documents\\GitHub\\gaolu\\MPC\\高炉\\0数据处理\\新输入输出模式\\new\\间隔1min平均.xlsx", engine='xlsxwriter') as writer:
    # 将其他 sheet 写入新文件
    new_sequence_1h_mean.to_excel(writer, sheet_name='1h_mean', index=False)


In [ ]:
C:\Users\haokw\Documents\GitHub\gaolu\MPC\高炉\0数据处理\新输入输出模式\new\获取时间戳.ipynb


In [10]:
# 间隔1min
new_sequence_60s = df_processed.iloc[::6][input_term]

new_sequence_60s.info()

# 将 df_combined 添加到新的 ExcelWriter 中
with pd.ExcelWriter(f"C:\\Users\\haokw\\Documents\\GitHub\\gaolu\\MPC\\高炉\\0数据处理\\新输入输出模式\\new\\间隔1min.xlsx", engine='xlsxwriter') as writer:
    # 将其他 sheet 写入新文件
    new_sequence_60s.to_excel(writer, sheet_name='60s', index=False)



<class 'pandas.core.frame.DataFrame'>
RangeIndex: 55098 entries, 0 to 330582
Data columns (total 37 columns):
 #   Column    Non-Null Count  Dtype         
---  ------    --------------  -----         
 0   主参数时间戳    55098 non-null  datetime64[ns]
 1   主参数时间戳2   55098 non-null  float64       
 2   富氧率       55098 non-null  float64       
 3   透气性指数     55098 non-null  float64       
 4   CO        55098 non-null  float64       
 5   H2        55098 non-null  float64       
 6   CO2       55098 non-null  float64       
 7   标准风速      55098 non-null  int64         
 8   富氧流量      55098 non-null  int64         
 9   冷风流量      55098 non-null  float64       
 10  鼓风动能      55098 non-null  int64         
 11  炉腹煤气量     55098 non-null  int64         
 12  炉腹煤气指数    55098 non-null  float64       
 13  理论燃烧温度    55098 non-null  int64         
 14  顶压        55098 non-null  float64       
 15  顶压2       55098 non-null  float64       
 16  顶压3       55098 non-null  float64       
 17  顶压4       5

In [ ]:
# 间隔10min
new_sequence_10min = df_processed.iloc[::60][input_term]

new_sequence_10min.info()

# 将 df_combined 添加到新的 ExcelWriter 中
with pd.ExcelWriter(f"C:\\Users\\haokw\\Documents\\GitHub\\gaolu\\MPC\\高炉\\0数据\\高频\\间隔10min.xlsx", engine='xlsxwriter') as writer:
    # 将其他 sheet 写入新文件
    new_sequence_10min.to_excel(writer, sheet_name='10min', index=False)



In [ ]:
# 间隔2min均值  2min = 120s  120/10 = 12
splice_ = 12
total_entries = len(df_params)
num_chunks = total_entries // splice_ + 1
dataframe = pd.DataFrame()
for i in range(num_chunks):
    start_idx = i * splice_
    end_idx = min((i + 1) * splice_, total_entries)
    print(i,start_idx,end_idx)
    # 获取子集
    subset_df = df_processed.iloc[start_idx:end_idx]
    # print(subset_df)
    subset_df_processed = subset_df.groupby(subset_df.index // splice_)[input_term].mean()
    # print(subset_df_processed)
    subset_df_processed['主参数时间戳2'] = df_processed.iloc[end_idx-1]['主参数时间戳2']
    subset_df_processed['主参数时间戳'] = df_processed.iloc[end_idx-1]['主参数时间戳']
    # print(subset_df_processed)
    # print('dbdfhbfdhb')
    dataframe = pd.concat([dataframe,subset_df_processed],axis=0)
    
    
new_sequence_2min_mean = dataframe.copy()
# 将 df_combined 添加到新的 ExcelWriter 中
with pd.ExcelWriter(f"C:\\Users\\haokw\\Documents\\GitHub\\gaolu\\MPC\\高炉\\0数据\\高频\\间隔2min均值.xlsx", engine='xlsxwriter') as writer:
    # 将其他 sheet 写入新文件
    new_sequence_2min_mean.to_excel(writer, sheet_name='2min_mean', index=False)


In [ ]:
# 间隔5min均值  5min = 300s  300/10 = 30
splice_ = 30
total_entries = len(df_params)
num_chunks = total_entries // splice_ + 1
dataframe = pd.DataFrame()
for i in range(num_chunks):
    start_idx = i * splice_
    end_idx = min((i + 1) * splice_, total_entries)
    print(i,start_idx,end_idx)
    # 获取子集
    subset_df = df_processed.iloc[start_idx:end_idx]
    # print(subset_df)
    subset_df_processed = subset_df.groupby(subset_df.index // splice_)[input_term].mean()
    # print(subset_df_processed)
    subset_df_processed['主参数时间戳2'] = df_processed.iloc[end_idx-1]['主参数时间戳2']
    subset_df_processed['主参数时间戳'] = df_processed.iloc[end_idx-1]['主参数时间戳']
    # print(subset_df_processed)
    # print('dbdfhbfdhb')
    dataframe = pd.concat([dataframe,subset_df_processed],axis=0)
    
    
new_sequence_5min_mean = dataframe.copy()
# 将 df_combined 添加到新的 ExcelWriter 中
with pd.ExcelWriter(f"C:\\Users\\haokw\\Documents\\GitHub\\gaolu\\MPC\\高炉\\0数据\\高频\\间隔5min均值.xlsx", engine='xlsxwriter') as writer:
    # 将其他 sheet 写入新文件
    new_sequence_5min_mean.to_excel(writer, sheet_name='5min_mean', index=False)


In [ ]:
# 间隔10min均值  10min = 600s  600/10 = 60
splice_ = 60
total_entries = len(df_params)
num_chunks = total_entries // splice_ + 1
dataframe = pd.DataFrame()
for i in range(num_chunks):
    start_idx = i * splice_
    end_idx = min((i + 1) * splice_, total_entries)
    print(i,start_idx,end_idx)
    # 获取子集
    subset_df = df_processed.iloc[start_idx:end_idx]
    # print(subset_df)
    subset_df_processed = subset_df.groupby(subset_df.index // splice_)[input_term].mean()
    # print(subset_df_processed)
    subset_df_processed['主参数时间戳2'] = df_processed.iloc[end_idx-1]['主参数时间戳2']
    subset_df_processed['主参数时间戳'] = df_processed.iloc[end_idx-1]['主参数时间戳']
    # print(subset_df_processed)
    # print('dbdfhbfdhb')
    dataframe = pd.concat([dataframe,subset_df_processed],axis=0)
    
    
new_sequence_10min_mean = dataframe.copy()
# 将 df_combined 添加到新的 ExcelWriter 中
with pd.ExcelWriter(f"C:\\Users\\haokw\\Documents\\GitHub\\gaolu\\MPC\\高炉\\0数据\\高频\\"+'new_sequence_10min_mean'+".xlsx", engine='xlsxwriter') as writer:
    # 将其他 sheet 写入新文件
    new_sequence_10min_mean.to_excel(writer, sheet_name='10min_mean', index=False)


In [ ]:
# 间隔60min均值
index = np.arange(15,7413,1)
len(index)

end_idx = 15*3600
time = 3600
dataframe = pd.DataFrame()

for i in range(len(index)-1):
    # if i < 7360:
    #     continue
    print(i,index[i])
    start_idx = df_processed[df_processed['主参数时间戳2'] < index[i]*time].index[-1]
    # print(start_idx,df_processed['主参数时间戳2'][start_idx])
    end_idx = df_processed[df_processed['主参数时间戳2'] < index[i+1]*time].index[-1]
    # print(end_idx,df_processed['主参数时间戳2'][end_idx])
    # 获取子集
    subset_df = df_processed.iloc[start_idx:end_idx]
    # print(subset_df.head(2))
    # print(subset_df.tail(2))
    # print('fdsahdrhgdfx hfxthdfghy j gh')
    subset_df_processed = pd.DataFrame(subset_df[input_term].mean()).T
    subset_df_processed['主参数时间戳2']=index[i+1]*time
    # print(subset_df_processed)
    dataframe = pd.concat([dataframe,subset_df_processed],axis=0)
    


    
print(dataframe)
new_sequence_1h_mean = dataframe.copy()
# 将 df_combined 添加到新的 ExcelWriter 中
with pd.ExcelWriter(f"C:\\Users\\haokw\\Documents\\GitHub\\gaolu\\MPC\\高炉\\0数据\\高频\\1h_mean.xlsx", engine='xlsxwriter') as writer:
    # 将其他 sheet 写入新文件
    new_sequence_1h_mean.to_excel(writer, sheet_name='1h_mean', index=False)


5270 5285
5271 5286
5272 5287
5273 5288
5274 5289
5275 5290
5276 5291
5277 5292
5278 5293
5279 5294
5280 5295
5281 5296
5282 5297
5283 5298
5284 5299
5285 5300
5286 5301
5287 5302
5288 5303
5289 5304
5290 5305
5291 5306
5292 5307
5293 5308
5294 5309
5295 5310
5296 5311
5297 5312
5298 5313
5299 5314
5300 5315
5301 5316
5302 5317
5303 5318
5304 5319
5305 5320
5306 5321
5307 5322
5308 5323
5309 5324
5310 5325
5311 5326
5312 5327
5313 5328
5314 5329
5315 5330
5316 5331
5317 5332
5318 5333
5319 5334
5320 5335
5321 5336
5322 5337
5323 5338
5324 5339
5325 5340
5326 5341
5327 5342
5328 5343
5329 5344
5330 5345
5331 5346
5332 5347
5333 5348
5334 5349
5335 5350
5336 5351
5337 5352
5338 5353
5339 5354
5340 5355
5341 5356
5342 5357
5343 5358
5344 5359
5345 5360
5346 5361
5347 5362
5348 5363
5349 5364
5350 5365
5351 5366
5352 5367
5353 5368
5354 5369
5355 5370
5356 5371
5357 5372
5358 5373
5359 5374
5360 5375
5361 5376
5362 5377
5363 5378
5364 5379
5365 5380
5366 5381
5367 5382
5368 5383
5369 5384


In [10]:
# 读取Excel文件
excel_path = f'C:\\Users\\haokw\\Documents\\GitHub\\gaolu\\MPC\\高炉\\0数据\\高频\\高频下采样到SI\\真实SI_时间戳s.xlsx'
df_sheet_yuansu = pd.read_excel(excel_path, sheet_name='Sheet1') 
print(df_sheet_yuansu.info())
print(df_sheet_yuansu.columns)


<class 'pandas.core.frame.DataFrame'>
RangeIndex: 16252 entries, 0 to 16251
Data columns (total 3 columns):
 #   Column   Non-Null Count  Dtype  
---  ------   --------------  -----  
 0   real时间戳  16252 non-null  float64
 1   时间戳s     16252 non-null  float64
 2   SI       16252 non-null  float64
dtypes: float64(3)
memory usage: 381.0 KB
None
Index(['real时间戳', '时间戳s', 'SI'], dtype='object')


In [11]:
df_sheet_yuansu.tail(5)


,real时间戳,时间戳s,SI
16247,7582.170556,27295814.0,0.373
16248,7583.230278,27299629.0,0.420
16249,7583.338611,27300019.0,0.437
16250,7583.476944,27300517.0,0.379
16251,7583.804444,27301696.0,0.353


In [13]:
index


array([  14.83972222,   14.98555556,   15.15805556, ..., 7583.33861111,
       7583.47694444, 7583.80444444])

In [23]:
# 间隔60min均值
index = np.array(df_sheet_yuansu['real时间戳'])
len(index)
err = 0
act = 0

dataframe = pd.DataFrame()
time =3600
for i in range(2,len(index)):
    # print(i, index[i], index[i]*time)
    start_idx = df_processed[df_processed['主参数时间戳2'] < (index[i]-1)*time].index[-1]
    # print('start:',start_idx,df_processed['主参数时间戳2'][start_idx])
    end_idx = df_processed[df_processed['主参数时间戳2'] < index[i]*time].index[-1]
    # print('end  :',end_idx,df_processed['主参数时间戳2'][end_idx])
    # 获取子集
    subset_df = df_processed.iloc[start_idx:end_idx]
    
    # print(end_idx-start_idx)
    if (end_idx-start_idx) < 355:
        err = err+1
        continue
    # print(subset_df.head(2))
    # print(subset_df.tail(2))
    # print('fdsahdrhgdfx hfxthdfghy j gh')
    subset_df_processed = pd.DataFrame(subset_df.mean()).T
    subset_df_processed['主参数时间戳2']=index[i]*time
    subset_df_processed['主参数时间戳3']=index[i]
    # print(subset_df_processed)
    dataframe = pd.concat([dataframe,subset_df_processed],axis=0)
    # break
    print(i, index[i], index[i]*time,end_idx-start_idx,act,err)
    act =act+1
print(act,err)


    
print(dataframe)





2 15.15805555555556 54569.000000000015 359 0 0
3 15.65222222222222 56347.99999999999 360 1 0
4 16.99944444444445 61198.00000000002 359 2 0
5 17.06611111111111 61437.99999999999 360 3 0
6 17.51611111111111 63057.99999999999 359 4 0
7 17.90166666666667 64446.000000000015 359 5 0
8 19.04666666666667 68568.00000000001 360 6 0
9 19.41805555555555 69904.99999999999 360 7 0
10 20.10583333333333 72380.99999999999 360 8 0
11 20.71888888888889 74588.00000000001 359 9 0
12 21.61472222222222 77812.99999999999 360 10 0
13 22.21333333333333 79967.99999999999 360 11 0
14 22.54388888888889 81158.0 360 12 0
15 23.33444444444444 84003.99999999999 360 13 0
16 23.90777777777778 86068.00000000001 359 14 0
17 25.905 93258.0 359 15 0
18 25.95222222222222 93427.99999999999 359 16 0
19 26.17444444444444 94227.99999999999 359 17 0
20 26.61888888888889 95828.0 360 18 0
21 26.93555555555556 96968.00000000001 360 19 0
22 28.39388888888889 102218.0 359 20 0
23 28.83 103788.0 359 21 0
24 29.175 105030.0 359 22 0
25 

In [24]:
new_sequence_1h_mean = dataframe.copy()
# 将 df_combined 添加到新的 ExcelWriter 中
with pd.ExcelWriter(f"C:\\Users\\haokw\\Documents\\GitHub\\gaolu\\MPC\\高炉\\0数据\\高频\\1h_mean_si.xlsx", engine='xlsxwriter') as writer:
    # 将其他 sheet 写入新文件
    new_sequence_1h_mean.to_excel(writer, sheet_name='1h_mean', index=False)
